Just the dataset differents from `gcp2pnet.train.main()`

Train successfully, 

potential problem: `args.frozen_weight=None` setting or not

In [1]:
import os
import random
from scipy import spatial
import networkx as nx
import torch
import numpy as np
from torch.utils.data import Dataset
from PIL import Image
import cv2
import glob
import scipy.io as io
from matplotlib import pyplot as plt

import argparse
import datetime
import random
import time
from pathlib import Path
from IPython.display import clear_output

import torch
from torch.utils.data import DataLoader, DistributedSampler

import os
from tensorboardX import SummaryWriter
import warnings
warnings.filterwarnings('ignore')

In [2]:
import sys

sys.path.insert(0, "../")

import gcp2pnet

from gcp2pnet.utils import DeNormalize, gen_random_scale_n
from gcp2pnet.engine import train_one_epoch, evaluate_crowd_no_overlap

# import gcp2pnet.misc as utils

from gcp2pnet.models.attention import SpatialAttention, ChannelwiseAttention

from gcp2pnet.models.pyrimid import (
    vgg_conv1_2, vgg_conv2_2, vgg_conv3_3, vgg_conv4_3, vgg_conv5_3,
    conv_1_2_hook, conv_2_2_hook, conv_3_3_hook, conv_4_3_hook, conv_5_3_hook,
    CPFE, SODModel, 
)

from gcp2pnet.models.matcher import HungarianMatcher_Crowd

from gcp2pnet.models.p2pnet import (
    RegressionModel, ClassificationModel, 
    generate_anchor_points, shift, 
    AnchorPoints, P2PNet, SetCriterion_Crowd, 
    build, build_eval,
)

from gcp2pnet.train import get_train_arguments

In [3]:
trimming_size               = 256
padding_size                = 0
learning_rate               = 1e-3 #learning rate for backgrond model
learning_rate_2             = 1e-4 #learning rate for detection head

label_dict = {'Fill': 1, '平べったいけど沈む': 1, '平べったくて浮く': 2, '詰まっているけど浮く': 2, 'Unfill': 2} 
label_type_count = 2+1

ROOT_DIR = r"/home/hwang/d/GoogleDrive/howcanoewang@gmail.com/SharedWithMe/GrainCounting"

training_data_root_dir      = f"{ROOT_DIR}/AnalysisData_small/"
training_data_root_dir2     = f"{ROOT_DIR}/AnalysisData/"
program_files_root_dir      = f"{ROOT_DIR}/programs/"
net_code_dir                = f"{program_files_root_dir}/CrowdCounting-P2PNet"
best_model_file             = f"{net_code_dir}/MultiLevelPyramidFeature_01/best_mae.pth"

####下記フォルダに、学習させたい画像と、Annotationデータをセットで配置します

training_set_root_data_dir                          = training_data_root_dir2 + "SetRootData/"  #こちらの各ディレクトリにjsonと画像ファイルを配置する

training_data_images_dir_before_trimming            = training_data_root_dir + "images/ImageBeforeTrimming/" #こちらに画像を配置する
evaluation_data_images_dir_before_trimming          = training_data_root_dir + "evaluation/ImageBeforeTrimming/" #こちらに画像を配置する
inference_input_path_before_trimming                = training_data_root_dir + "inference/Input/image_before_trimming" #こちらに画像を配置する

training_data_training_label_dir                    = training_data_root_dir + "labels_Training" #こちらにjsonファイルを配置する
training_data_evaluation_label_dir                  = training_data_root_dir + "labels_Evaluation" #こちらにjsonファイルを配置する

####学習用パッチ画像を生成すると更新される
training_data_images_dir                            = training_data_root_dir + "images/" #こちらにパッチ画像が生成される
evaluation_data_images_dir                          = training_data_root_dir + "evaluation/" #こちらにパッチ画像が生成される

training_data_label_dir_before_trimming             = training_data_root_dir + "labels_Training/only_coordinate/ImageBeforeTrimming/" #こちらにjsonを変換したtxtが生成される
evaluation_data_label_dir_before_trimming           = training_data_root_dir + "labels_Evaluation/only_coordinate/ImageBeforeTrimming/" #こちらにjsonを変換したtxtが生成される

training_data_points_dir                            = training_data_root_dir + "labels_Training/only_coordinate/" #こちらにtxtファイルをパッチサイズで切り出したファイルが生成される
evaluation_data_points_dir                          = training_data_root_dir + "labels_Evaluation/only_coordinate/" #こちらにtxtファイルをパッチサイズで切り出したファイルが生成される

training_list_path                                  = training_data_root_dir + "training_list.txt" #こちらに画像⇔txtの対応ファイルが生成される
evaluating_list_path                                = training_data_root_dir + "evaluation_list.txt" #こちらに画像⇔txtの対応ファイルが生成される

#推論用パッチ画像を生成すると更新される
# inference_input_path                                = training_data_root_dir + "inference/Input"#こちらにパッチ画像が生成される
# inference_input_path                                = "../data/inference"
# inference_output_path                               = training_data_root_dir + "inference/Output"#こちらに推論結果が生成される
# inference_output_path                               = "./outputs/exp_v3"

inference_integrated_image_file_path                = training_data_root_dir + "inference/Output/Integrated"#パッチ統合後画像が生成される

# Datasets 

In [4]:
###############
# datasets.py
###############

class SHHA(Dataset):
    def __init__(self, data_root, transform=None, train=False, patch=False, flip=False):
        self.root_path = data_root

        #############################
        #   Global variables DANGER #
        #############################
        self.train_lists = training_list_path ## rice grainsの場合
        self.eval_lists = evaluating_list_path ## rice grainsの場合

        # there may exist multiple list files
        # e.g. 
        # ```
        # content/drive/MyDrive/research/tanashi2022/GrainCounting/AnalysisData_small/images/20230209_08_G_a_v05_h11.JPG,/content/drive/MyDrive/research/tanashi2022/GrainCounting/AnalysisData_small/labels_Training/only_coordinate/20230209_08_G_a_v05_h11.txt
        # /content/drive/MyDrive/research/tanashi2022/GrainCounting/AnalysisData_small/images/20221227_07_R_v05_h01.JPG,/content/drive/MyDrive/research/tanashi2022/GrainCounting/AnalysisData_small/labels_Training/only_coordinate/20221227_07_R_v05_h01.txt
        # if train:
        #     self.img_list_file = [name.split(',') for name in open(self.train_lists).read().splitlines()]
        # else:
        #     self.img_list_file = [name.split(',') for name in open(self.eval_lists).read().splitlines()]

        # self.img_list = self.img_list_file

        if train:
            self.img_list = train_img_list
        else:
            self.img_list = eval_img_list

        # number of samples
        self.nSamples = len(self.img_list)

        self.transform = transform
        self.train = train
        self.patch = patch
        self.flip = flip

    def __len__(self):
        return self.nSamples

    def __getitem__(self, index):
        assert index <= len(self), 'index range error'


        img_path = self.img_list[index][0]
        gt_path = self.img_list[index][1]

        # load image and ground truth
        img, point, labels = load_data((img_path, gt_path), self.train)

        # applu augumentation
        if self.transform is not None:
            img = self.transform(img)

        if self.train:
            # data augmentation -> random scale
            scale_range = [0.5, 1.3]
            min_size = min(img.shape[1:])
            scale = random.uniform(*scale_range)
            # scale the image and points
            if scale * min_size > trimming_size:
                img = torch.nn.functional.upsample_bilinear(img.unsqueeze(0), scale_factor=scale).squeeze(0)
                point *= scale
        # random crop augumentaiton
        if self.train and self.patch:
            img, point,labels = random_crop(img, point, labels)

            for i, _ in enumerate(point):  #03/21 debug
                point[i] = torch.Tensor(point[i])
                labels[i] = torch.Tensor(labels[i])

        # random flipping
        if random.random() > 0.5 and self.train and self.flip:
            # random flip
            img = torch.Tensor(img[:, :, :, ::-1].copy())

            for i, _ in enumerate(point):
                point[i][:, 0] = trimming_size - point[i][:, 0]

        if not self.train:
            point = [point]
            labels = [labels]

        img = torch.Tensor(img)
        # pack up related infos
        target = [{} for i in range(len(point))]

        for i, _ in enumerate(point):  #03/21 debug
            target[i]['point'] = torch.Tensor(point[i])

            if len(labels[0]) > 1:
                target[i]['labels'] = torch.Tensor(labels[i].flatten()).long()
            else:
                target[i]['labels'] = torch.Tensor(labels[i]).long()

            image_id_1 = int(img_path.split('/')[-1].split('.')[0][5:7])
            image_id_1 = torch.Tensor([image_id_1]).long()
            #
            image_id_2 = int(img_path.split('/')[-1].split('.')[0][5:7])
            image_id_2 = torch.Tensor([image_id_2]).long()

            target[i]['image_id_1'] = image_id_1
            target[i]['image_id_2'] = image_id_2

        return img, target


def load_data(img_gt_path, train):
    img_path, gt_path = img_gt_path

    # load the images
    img = cv2.imread(img_path)

    if not img is None:
        img = Image.fromarray(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))

    # load ground truth points
    points = []
    labels = []
    #
    pts = open(gt_path).read().splitlines()
    for pt_0 in pts:
        pt = eval(pt_0)
        x = float(pt[0])#/2
        y = float(pt[1])#/2
        label = float(pt[2])
        points.append([x, y])
        labels.append([label])

    return img, np.array(points), np.array(labels)


# random crop augumentation
def random_crop(img, den, labels, num_patch=1):
    half_h = trimming_size
    half_w = trimming_size
    result_img = np.zeros([num_patch, img.shape[0], half_h, half_w])
    result_den = []
    result_label = []
    # crop num_patch for each image
    for i in range(num_patch):
        start_h = random.randint(0, img.size(1) - half_h)
        start_w = random.randint(0, img.size(2) - half_w)
        end_h = start_h + half_h
        end_w = start_w + half_w
        # copy the cropped rect
        result_img[i] = img[:, start_h:end_h, start_w:end_w]
        # copy the cropped points
        idx = (den[:, 0] >= start_w) & (den[:, 0] <= end_w) & (den[:, 1] >= start_h) & (den[:, 1] <= end_h)
        # shift the corrdinates
        record_den = den[idx]
        record_label = labels[idx]
        record_den[:, 0] -= start_w
        record_den[:, 1] -= start_h

        result_den.append(record_den)
        result_label.append(record_label)

    return result_img, result_den, result_label

###############
# load_data.py
###############
import torchvision.transforms as standard_transforms

# DeNormalize used to get original images
class DeNormalize(object):
    def __init__(self, mean, std):
        self.mean = mean
        self.std = std

    def __call__(self, tensor):
        for t, m, s in zip(tensor, self.mean, self.std):
            t.mul_(s).add_(m)
        return tensor

def loading_data(data_root):
    # the pre-proccssing transform
    transform = standard_transforms.Compose([
        standard_transforms.ToTensor(),
        standard_transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                    std=[0.229, 0.224, 0.225]),
    ])
    # create the training dataset
    train_set = SHHA(data_root, train=True, transform=transform, patch=True, flip=True)
    # create the validation dataset
    val_set = SHHA(data_root, train=False, transform=transform)

    return train_set, val_set

In [5]:
train_img_list = []
solid_path = '/content/drive/MyDrive/research/tanashi2022/GrainCounting/AnalysisData_small/'

with open(training_list_path, 'r') as f:
    for line in f.readlines():
        # one line: `path/img_name.JPG,path/img_name.txt`
        split_content = line.split(',')

        new_split_content = [item.replace(solid_path, training_data_root_dir ).replace('\n', '') for item in split_content]

        train_img_list.append( new_split_content )

In [6]:
eval_img_list = []
solid_path = '/content/drive/MyDrive/research/tanashi2022/GrainCounting/AnalysisData_small/'

with open(evaluating_list_path, 'r') as f:
    for line in f.readlines():
        # one line: `path/img_name.JPG,path/img_name.txt`
        split_content = line.split(',')

        new_split_content = [item.replace(solid_path, training_data_root_dir ).replace('\n', '') for item in split_content]

        eval_img_list.append( new_split_content )

In [7]:
train_set, val_set = loading_data( training_data_root_dir )

In [11]:
args = get_train_arguments()

args.frozen_weights = None
args.output_dir = "./outputs"
args.run_name = "exp_v3"

Modifed `train.main()`

In [9]:
import yaml

In [12]:
os.environ["CUDA_VISIBLE_DEVICES"] = '{}'.format(args.gpu_id)

run_output_dir = Path(args.output_dir) / args.run_name
run_output_dir.mkdir(parents=True, exist_ok=True)

weights_dir = run_output_dir / 'weights'
weights_dir.mkdir(parents=True, exist_ok=True)
best_model_file = weights_dir / "best_mae.pth"

args.checkpoints_dir = weights_dir
args.vis_dir = run_output_dir

args.tensorboard_dir = run_output_dir / "tensorboard_logs"
args.tensorboard_dir.mkdir(parents=True, exist_ok=True)

latest_ckpt = weights_dir / 'latest.pth'

if latest_ckpt.exists() and args.resume == '':
    user_input = input(f"Found previous checkpoints at [{latest_ckpt}], resume training [Y/Yes] or retrain [N/No]? ").lower()
    if user_input in ['y', 'yes']:
        args.resume = str(latest_ckpt)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
args.device = device

# create the logging file
run_log_name =  run_output_dir / 'args.yaml'
with open(run_log_name, 'w', encoding="utf-8") as f:
    yaml.dump({k: str(v) if isinstance(v, Path) else v for k, v in vars(args).items()}, f)

gcp2pnet.utils.fix_seed(args.seed)

# get the P2PNet model
model, criterion = gcp2pnet.models.p2pnet.build(args, training=True)

# move to GPU
model.to(args.device)
criterion.to(args.device)

model_without_ddp = model

n_parameters = sum(p.numel() for p in model.parameters() if p.requires_grad)
print('number of params:', n_parameters)
# use different optimation params for different parts of the model
param_dicts = [
    {"params": [p for n, p in model_without_ddp.named_parameters() if "fpn" not in n and p.requires_grad]},
    {
        "params": [p for n, p in model_without_ddp.named_parameters() if "fpn" in n and p.requires_grad],
        "lr": args.lr_fpn,
    },
]
# Adam is used by default
optimizer = torch.optim.Adam(param_dicts, lr=args.lr)
lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer, args.lr_drop)

# create the training and valiation set
# train_set, val_set = loading_data(args.data_root)  # args_dataroot = training_data_root_dir
# train_set, val_set = datasets.loading_dataset( args.dataset_folder )
# label_dict, class_n = datasets.loading_label_dict( args.dataset_folder ) 
class_n = 2

# create the sampler used during training
sampler_train = torch.utils.data.RandomSampler(train_set)
sampler_val = torch.utils.data.SequentialSampler(val_set)

batch_sampler_train = torch.utils.data.BatchSampler(
    sampler_train, args.batch_size, drop_last=True)

# the dataloader for training
data_loader_train = DataLoader(train_set, batch_sampler=batch_sampler_train,
                                collate_fn=gcp2pnet.misc.collate_fn_crowd, 
                                num_workers=args.num_workers)

data_loader_val = DataLoader(val_set, 1, sampler=sampler_val,
                                drop_last=False, collate_fn=gcp2pnet.misc.collate_fn_crowd, 
                                num_workers=args.num_workers)

if args.frozen_weights is not None:
    # checkpoint = torch.load(args.frozen_weights, map_location='cpu')
    checkpoint = torch.load(args.frozen_weights, weights_only=False, map_location=device)
    model_without_ddp.detr.load_state_dict(checkpoint['model'])

# resume the weights and training state if exists
if args.resume:
    print("Resume previous checkpoints, continue training")
    # checkpoint = torch.load(args.resume, map_location='cpu')
    checkpoint = torch.load(args.resume, weights_only=False, map_location=device)
    model_without_ddp.load_state_dict(checkpoint['model'], strict=False)
    if not args.eval and \
        'optimizer' in checkpoint and \
        'lr_scheduler' in checkpoint and \
        'epoch' in checkpoint:

        optimizer.load_state_dict(checkpoint['optimizer'])
        lr_scheduler.load_state_dict(checkpoint['lr_scheduler'])
        args.start_epoch = checkpoint['epoch'] + 1
        print(checkpoint['epoch'] + 1)

gcp2pnet.utils.print_args(args)

number of params: 15053426
================================================== Training Arguments ==================================================
                  lr: 0.001
              lr_fpn: 0.0001
          batch_size: 1
        weight_decay: 0.0001
              epochs: 100
             lr_drop: 2000000
       clip_max_norm: 0.1
      frozen_weights: None
      set_cost_class: 0.5
      set_cost_point: 0.5
     point_loss_coef: 0.02
            eos_coef: 0.01
           threshold: 0.5
                 row: 2
                line: 2
      dataset_folder: ./data/dataset
          output_dir: ./outputs
            run_name: exp_v3
                seed: 42
              resume: 
         start_epoch: 0
                eval: False
         num_workers: 1
           eval_freq: 5
              gpu_id: 0
     checkpoints_dir: outputs/exp_v3/weights
             vis_dir: outputs/exp_v3
     tensorboard_dir: outputs/exp_v3/tensorboard_logs
              device: cuda


In [14]:
#######################
print("Start training")
#######################

start_time = time.time()

# the logger writer
writer = SummaryWriter(args.tensorboard_dir)

# save the performance during the training
mae = []
mse = []
step = 0

# training starts here
for epoch in range(args.start_epoch, args.epochs):

    # run evaluation
    if epoch % args.eval_freq == 0:
        t1 = time.time()
        result = gcp2pnet.engine.evaluate_crowd_no_overlap(model, data_loader_val, device, class_n, args.threshold)
        t2 = time.time()


        # save the best model since begining
        if epoch > 0:
            if np.min(mse) > result[1]:
            # if abs(np.min(mae) - result[0]) < 0.01:
                checkpoint_best_path = best_model_file
                torch.save({
                    'model': model_without_ddp.state_dict(),
                }, checkpoint_best_path)
                print("===updated best model===")

        mae.append(result[0])
        mse.append(result[1])

        print("mae list:", np.asarray(mae))
        print("mse list:", np.asarray(mse))

        # print the evaluation results
        print('=======================================test=======================================')
        print("mae:", result[0], "mse:", result[1], "time:", t2 - t1, "best mse:", np.min(mse), )
        # with open(run_log_name, "a") as log_file:
            # log_file.write("mae:{}, mse:{}, time:{}, best mae:{}".format(result[0],result[1], t2 - t1, np.min(mae)))
            # log_file.write("mae:{}, mse:{}, time:{}, best mse:{}".format(result[0],result[1], t2 - t1, np.min(mse)))
        # recored the evaluation results
        if writer is not None:
            writer.add_scalar('metric/mae', result[0], step)
            writer.add_scalar('metric/mse', result[1], step)
            step += 1

    t1 = time.time()
    stat = gcp2pnet.engine.train_one_epoch(model, criterion, data_loader_train, optimizer, device, epoch, args.clip_max_norm)

    # record the training states after every epoch
    if writer is not None:
        # with open(run_log_name, "a") as log_file:
        #     log_file.write("loss/loss@{}: {}".format(epoch, stat['loss']))
        #     log_file.write("loss/loss_ce@{}: {}".format(epoch, stat['loss_ce']))
        print(f"epoch: {epoch} with loss= {stat['loss']} loss_ce={stat['loss_ce']}")
        writer.add_scalar('loss/loss', stat['loss'], epoch)
        writer.add_scalar('loss/loss_ce', stat['loss_ce'], epoch)

    t2 = time.time()
    print('[ep %d][lr %.7f][%.2fs]' % (epoch, optimizer.param_groups[0]['lr'], t2 - t1))
    # with open(run_log_name, "a") as log_file:
    #     log_file.write('[ep %d][lr %.7f][%.2fs]' % (epoch, optimizer.param_groups[0]['lr'], t2 - t1))
    
    # change lr according to the scheduler
    lr_scheduler.step()
    
    # save latest weights every epoch
    if not os.path.exists(args.checkpoints_dir):
        os.makedirs(args.checkpoints_dir)
    checkpoint_latest_path = os.path.join(args.checkpoints_dir, 'latest.pth')

    torch.save({
        'model': model_without_ddp.state_dict(),
    }, checkpoint_latest_path)

# total time for training
total_time = time.time() - start_time
total_time_str = str(datetime.timedelta(seconds=int(total_time)))
print('Training time {}'.format(total_time_str))

Start training
mae list: [5.41035259]
mse list: [5.64198174]
=======================================test=======================================
mae: 5.4103525881470365 mse: 5.641981743102097 time: 3.2907180786132812 best mse: 5.641981743102097
Averaged stats: lr: 0.001000  loss: 0.1697 (0.4484)  loss_ce: 0.1581 (0.3588)  loss_points: 0.0116 (0.0896)  loss_ce_unscaled: 0.0527 (0.1196)  loss_points_unscaled: 0.5801 (4.4778)
epoch: 0 with loss= 0.4484020658338469 loss_ce=0.3588462857419966
[ep 0][lr 0.0010000][26.06s]
Averaged stats: lr: 0.001000  loss: 0.1548 (0.2211)  loss_ce: 0.1431 (0.2080)  loss_points: 0.0117 (0.0130)  loss_ce_unscaled: 0.0477 (0.0693)  loss_points_unscaled: 0.5842 (0.6521)
epoch: 1 with loss= 0.22108532990751634 loss_ce=0.20804372399563795
[ep 1][lr 0.0010000][25.84s]
Averaged stats: lr: 0.001000  loss: 0.1272 (0.2109)  loss_ce: 0.1126 (0.1977)  loss_points: 0.0145 (0.0132)  loss_ce_unscaled: 0.0375 (0.0659)  loss_points_unscaled: 0.7268 (0.6602)
epoch: 2 with loss

KeyboardInterrupt: 